# Début

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
#importing datas
from google.colab import drive
drive.mount('/content/drive')
stock_price = pd.read_csv('/content/drive/MyDrive/CAC40.csv')

stock_price.info()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       256 non-null    object 
 1   Open       256 non-null    float64
 2   High       256 non-null    float64
 3   Low        256 non-null    float64
 4   Close      256 non-null    float64
 5   Adj Close  256 non-null    float64
 6   Volume     256 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 14.1+ KB


In [ ]:
stock_price.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-10-26,5383.439941,5459.609863,5375.259766,5455.399902,5455.399902,90950700
1,2017-10-27,5470.339844,5513.529785,5470.240234,5494.129883,5494.129883,95585200
2,2017-10-30,5488.330078,5507.560059,5487.290039,5493.629883,5493.629883,67428800
3,2017-10-31,5492.629883,5511.250000,5489.410156,5503.290039,5503.290039,75164700
4,2017-11-01,5524.379883,5536.399902,5514.290039,5514.290039,5514.290039,75303700


In [ ]:
stock_price.index = pd.to_datetime(stock_price['Date'])

In [ ]:
stock_price = stock_price.drop('Date', axis = 1)

In [ ]:
stock_price.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-10-26,5383.439941,5459.609863,5375.259766,5455.399902,5455.399902,90950700
2017-10-27,5470.339844,5513.529785,5470.240234,5494.129883,5494.129883,95585200
2017-10-30,5488.330078,5507.560059,5487.290039,5493.629883,5493.629883,67428800
2017-10-31,5492.629883,5511.250000,5489.410156,5503.290039,5503.290039,75164700
2017-11-01,5524.379883,5536.399902,5514.290039,5514.290039,5514.290039,75303700


In [ ]:
data = stock_price.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis = 1)

In [ ]:
data

,Adj Close
Date,
2017-10-26,5455.399902
2017-10-27,5494.129883
2017-10-30,5493.629883
2017-10-31,5503.290039
2017-11-01,5514.290039
...,...
2018-10-22,5053.310059
2018-10-23,4967.689941
2018-10-24,4953.089844


# Indicateurs techniques

In [ ]:
#Moyenne mobile
def movingAverage(tab, N):
    ma_tab=[]
    for t in range(N, len(tab)+1):
        ma=np.mean(tab[t-N:t])
        ma_tab.append(ma)
    return ma_tab

In [ ]:
#Ecart-type
def standardDeviation(tab, N):
    sd_tab =[]
    for t in range(N,len(tab)+1):
        sd=np.std(tab[t-N:t])
        sd_tab.append(sd)
    return sd_tab

In [ ]:
def stochasticOscillator(tab, N):
    so_tab=[]
    for t in range(N, len(tab)+1):
        High = np.amax(tab[t-N:t])
        Low = np.amin(tab[t-N:t])
        P = tab[t-1]
        so = 100*(P-Low)/(High-Low)
        so_tab.append(so)
    return so_tab

In [ ]:
def relativeStrengthIndex(tab, N):
    rsi_tab = []
    for t in range(N,len(tab)+1):
        vals = tab[t-N:t]
        Un = 0
        nbUn = 0
        Dn = 0
        nbDn = 0
        for i in range(N-1):
            dif = vals[i+1]-vals[i]
            if (dif>0):
                Un +=dif
                nbUn +=1
            else:
                Dn +=abs(dif)
                nbDn +=1
        if (nbDn==0):
            rsi = 0.0
        elif (nbUn==0):
            rsi = 0.0
        else:
            Un /=nbUn
            Dn /=nbDn
            rsi = 100*(1-1/(1+Un/Dn))
        rsi_tab.append(rsi)
    return rsi_tab

In [ ]:
prices = stock_price['Close']
dates = stock_price.index

returns = np.array(prices.pct_change()[1:])
prices = np.array(prices)
b1 = np.quantile(returns, 0.3)
b2 = np.quantile(returns, 0.7)
b3 = np.quantile(returns, 0.5)

In [ ]:
def classifyMovements(tab, b1, b2):
    tab_ret = []
    for i in range(tab.size):
        if (tab[i]<=b1):
            tab_ret.append(1)
        elif (tab[i]>=b2):
            tab_ret.append(0)
        else:
            tab_ret.append(-1)
    return tab_ret

In [ ]:
def classifyMovements2(tab, b):
    tab_ret = []
    for i in range(tab.size):
        if (tab[i]<=b):
            tab_ret.append(1)
        else:
            tab_ret.append(-1)
    return tab_ret

In [ ]:
def encode_class(s_class):
  if (s_class == 1):
    return [1,0,0]
  elif (s_class == 0):
    return [0,1,0]
  elif (s_class == -1):
    return [0,0,1]

def encode_class2(s_class):
  if (s_class == 1):
    return [1,0]
  elif (s_class == -1):
    return [0,1]

# Préparation du dataframe avec 3 classes

In [ ]:
N=5

#Adding column Class

tab_cls = classifyMovements(returns, b1, b2)[N-2:]

lst_cls = list(tab_cls)
#Adding technical 

tab_ma = movingAverage(prices, N)
tab_sd = standardDeviation(prices, N)
tab_so = stochasticOscillator(prices, N)
tab_rsi = relativeStrengthIndex(prices, N)

In [ ]:
myData_3class = pd.DataFrame()
myData_3class['MA'] = tab_ma
myData_3class['SD'] = tab_sd
myData_3class['SO'] = tab_so
myData_3class['RSI'] = tab_rsi
myData_3class['CLS'] = tab_cls

In [ ]:
y_encoded_3class = np.array([encode_class(y) for y in list(tab_cls)])

# Pre-processing des données 3 classes



In [ ]:
data3 = myData_3class.drop('CLS', axis = 1)
target3 = y_encoded_3class

In [ ]:
from sklearn.model_selection import train_test_split

X_train3classes, X_test3classes, y_train3classes, y_test3classes = train_test_split(data3, target3, test_size = 0.2, random_state = 1234)

In [ ]:
y_train3classes

array([[1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0,

# Normalisation min-max 3 classes

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minmax3 = MinMaxScaler()
minmax2 = MinMaxScaler()

In [ ]:
#Normalization pour 3 classes

minmax3 = minmax3.fit(X_train3classes)
X_train3classes_norm = minmax3.transform(X_train3classes)
X_test3classes_norm = minmax3.transform(X_test3classes)

# Standardization 3 classes

In [ ]:
from sklearn.preprocessing import StandardScaler
standard3 = StandardScaler()

In [ ]:
#Standardization pour 3 classes

standard3 = standard3.fit(X_train3classes)
X_train3classes_norm = standard3.transform(X_train3classes)
X_test3classes_norm = standard3.transform(X_test3classes)
y_train3classes_norm = y_train3classes
y_test3classes_norm = y_test3classes

# 3 class Neural network

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as tfl

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

class Neural_network(Model):
    
    def __init__(self, var_param_list, target_param_list):
        
        # initialisation en tant que Model keras
        super(Neural_network, self).__init__()

        # couche entrée + encodeur 
        self.var_param_list = var_param_list
        self.var = tf.keras.Sequential(self.var_param_list)
        
        # couche décodeur + sortie
        self.target_param_list = target_param_list
        self.target = tf.keras.Sequential(self.target_param_list)

    def call(self, x):
        
        var = self.var(x)
        target = self.target(var)
        
        return target

In [ ]:
# initialisation des couches du réseau de neurone
nn = Neural_network([tfl.Dense(4, input_dim=4, activation='sigmoid')], # nombre de neurones = nombre de features (input = x)]
                    
                     [tfl.Dense(3, activation='softmax')]) # nombre de neurones = nombre de classes (output = 3)


# définition de l'optimizer
learning_rate = 3e-5 # learning rate
learning_rate = 0.01 # learning rate
loss = 'binary_crossentropy' # définition de la fonction de perte
loss = 'categorical_crossentropy'
optimizer = Adam(learning_rate=learning_rate)
metrics = ['accuracy']
nn.compile(optimizer=optimizer, loss=loss, metrics = metrics)

In [ ]:
# fit du modèle
epochs = 50
history = nn.fit(
    x=X_train3classes, y=y_train3classes, 
    epochs=200, 
    batch_size=10)

Epoch 1/200
21/21 [==============================] - 0s 2ms/step - loss: 1.1248 - accuracy: 0.3284
Epoch 2/200
21/21 [==============================] - 0s 2ms/step - loss: 1.0880 - accuracy: 0.4030
Epoch 3/200
21/21 [==============================] - 0s 3ms/step - loss: 1.0945 - accuracy: 0.4030
Epoch 4/200
21/21 [==============================] - 0s 2ms/step - loss: 1.0900 - accuracy: 0.4030
Epoch 5/200
21/21 [==============================] - 0s 2ms/step - loss: 1.0895 - accuracy: 0.4030
Epoch 6/200
21/21 [==============================] - 0s 2ms/step - loss: 1.0900 - accuracy: 0.4030
Epoch 7/200
21/21 [==============================] - 0s 2ms/step - loss: 1.0898 - accuracy: 0.4030
Epoch 8/200
21/21 [==============================] - 0s 3ms/step - loss: 1.0911 - accuracy: 0.4030
Epoch 9/200
21/21 [==============================] - 0s 2ms/step - loss: 1.0900 - accuracy: 0.4030
Epoch 10/200
21/21 [==============================] - 0s 2ms/step - loss: 1.0927 - accuracy: 0.4030
Epoch 11/

In [ ]:
score=nn.evaluate(X_test3classes, y_test3classes, verbose = 2)
score[1]

2/2 - 0s - loss: 1.0975 - accuracy: 0.3529 - 148ms/epoch - 74ms/step


0.3529411852359772

# 3 class Neural network norm



In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as tfl

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

class Neural_network(Model):
    
    def __init__(self, var_param_list, target_param_list):
        
        # initialisation en tant que Model keras
        super(Neural_network, self).__init__()

        # couche entrée + encodeur 
        self.var_param_list = var_param_list
        self.var = tf.keras.Sequential(self.var_param_list)
        
        # couche décodeur + sortie
        self.target_param_list = target_param_list
        self.target = tf.keras.Sequential(self.target_param_list)

    def call(self, x):
        
        var = self.var(x)
        target = self.target(var)
        
        return target

In [ ]:
# initialisation des couches du réseau de neurone
nn = Neural_network([tfl.Dense(4, input_dim=4, activation='sigmoid')], # nombre de neurones = nombre de features (input = x)]
                     [tfl.Dense(3, activation='softmax')]) # nombre de neurones = nombre de classes (output = 3)


# définition de l'optimizer
learning_rate = 3e-5 # learning rate
learning_rate = 0.01 # learning rate
loss = 'binary_crossentropy' # définition de la fonction de perte
loss = 'categorical_crossentropy'
optimizer = Adam(learning_rate=learning_rate)
metrics = ['accuracy']
nn.compile(optimizer=optimizer, loss=loss, metrics = metrics)

In [ ]:
# fit du modèle
epochs = 50
history = nn.fit(
    x=X_train3classes_norm, y=y_train3classes_norm, 
    epochs=200, 
    batch_size=10)

Epoch 1/200
21/21 [==============================] - 0s 2ms/step - loss: 1.1147 - accuracy: 0.3333
Epoch 2/200
21/21 [==============================] - 0s 2ms/step - loss: 1.0119 - accuracy: 0.4577
Epoch 3/200
21/21 [==============================] - 0s 2ms/step - loss: 0.9734 - accuracy: 0.4776
Epoch 4/200
21/21 [==============================] - 0s 2ms/step - loss: 0.9419 - accuracy: 0.5224
Epoch 5/200
21/21 [==============================] - 0s 2ms/step - loss: 0.9186 - accuracy: 0.5423
Epoch 6/200
21/21 [==============================] - 0s 2ms/step - loss: 0.8961 - accuracy: 0.5572
Epoch 7/200
21/21 [==============================] - 0s 2ms/step - loss: 0.8797 - accuracy: 0.5224
Epoch 8/200
21/21 [==============================] - 0s 2ms/step - loss: 0.8673 - accuracy: 0.5473
Epoch 9/200
21/21 [==============================] - 0s 2ms/step - loss: 0.8557 - accuracy: 0.5622
Epoch 10/200
21/21 [==============================] - 0s 2ms/step - loss: 0.8484 - accuracy: 0.6020
Epoch 11/

In [ ]:
score=nn.evaluate(X_test3classes_norm, y_test3classes_norm, verbose = 2)
score[1]

2/2 - 0s - loss: 0.7270 - accuracy: 0.6471 - 126ms/epoch - 63ms/step


0.6470588445663452

# Préparation du dataframe avec 2 classes

In [ ]:
N=5

#Adding column Class

tab_cls = classifyMovements2(returns, b3)[N-2:]

lst_cls = list(tab_cls)
#Adding technical 

tab_ma = movingAverage(prices, N)
tab_sd = standardDeviation(prices, N)
tab_so = stochasticOscillator(prices, N)
tab_rsi = relativeStrengthIndex(prices, N)

In [ ]:
myData_2class = pd.DataFrame()
myData_2class['MA'] = tab_ma
myData_2class['SD'] = tab_sd
myData_2class['SO'] = tab_so
myData_2class['RSI'] = tab_rsi
myData_2class['CLS'] = tab_cls

In [ ]:
y_encoded_2class = np.array([encode_class2(y) for y in list(tab_cls)])

# Pre-processing des données 2 classes



In [ ]:
data2 = myData_2class.drop('CLS', axis = 1)
target2 = y_encoded_2class

In [ ]:
from sklearn.model_selection import train_test_split

X_train2classes, X_test2classes, y_train2classes, y_test2classes = train_test_split(data2, target2, test_size = 0.2, random_state = 1234)

In [ ]:
y_train2classes

array([[1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [0,

# Normalisation min-max 2 classes

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minmax2 = MinMaxScaler()

In [ ]:
#Normalization pour 3 classes

minmax2 = minmax2.fit(X_train2classes)
X_train2classes_norm = minmax2.transform(X_train2classes)
X_test2classes_norm = minmax2.transform(X_test2classes)

# Standardization 2 classes

In [ ]:
from sklearn.preprocessing import StandardScaler
standard2 = StandardScaler()

In [ ]:
#Standardization pour 3 classes

standard2 = standard2.fit(X_train2classes)
X_train2classes_norm = standard2.transform(X_train2classes)
X_test2classes_norm = standard2.transform(X_test2classes)
y_train2classes_norm = y_train2classes
y_test2classes_norm = y_test2classes

# 2 class Neural network

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as tfl

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

class Neural_network(Model):
    
    def __init__(self, var_param_list, target_param_list):
        
        # initialisation en tant que Model keras
        super(Neural_network, self).__init__()

        # couche entrée + encodeur 
        self.var_param_list = var_param_list
        self.var = tf.keras.Sequential(self.var_param_list)
        
        # couche décodeur + sortie
        self.target_param_list = target_param_list
        self.target = tf.keras.Sequential(self.target_param_list)

    def call(self, x):
        
        var = self.var(x)
        target = self.target(var)
        
        return target

In [ ]:
# initialisation des couches du réseau de neurone
nn2 = Neural_network([tfl.Dense(4, input_dim=4, activation='sigmoid')], # nombre de neurones = nombre de features (input = x)]
                    
                     [tfl.Dense(2, activation='softmax')]) # nombre de neurones = nombre de classes (output = 3)


# définition de l'optimizer
learning_rate2 = 3e-5 # learning rate
learning_rate2 = 0.01 # learning rate
loss2 = 'binary_crossentropy' # définition de la fonction de perte
#loss2 = 'categorical_crossentropy'
optimizer2 = Adam(learning_rate=learning_rate2)
metrics2 = ['accuracy']
nn2.compile(optimizer=optimizer2, loss=loss2, metrics = metrics2)

In [ ]:
# fit du modèle
epochs = 50
history = nn2.fit(
    x=X_train2classes, y=y_train2classes, 
    epochs=200, 
    batch_size=10)

Epoch 1/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6943 - accuracy: 0.4876
Epoch 2/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6930 - accuracy: 0.5174
Epoch 3/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6933 - accuracy: 0.5174
Epoch 4/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6928 - accuracy: 0.5174
Epoch 5/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6926 - accuracy: 0.5174
Epoch 6/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6930 - accuracy: 0.5174
Epoch 7/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6930 - accuracy: 0.5174
Epoch 8/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5174
Epoch 9/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6935 - accuracy: 0.5174
Epoch 10/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6930 - accuracy: 0.5174
Epoch 11/

In [ ]:
score=nn2.evaluate(X_test2classes, y_test2classes, verbose = 2)
score[1]

2/2 - 0s - loss: 0.7016 - accuracy: 0.4118 - 124ms/epoch - 62ms/step


0.4117647111415863

# 3 class Neural network norm



In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as tfl

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

class Neural_network(Model):
    
    def __init__(self, var_param_list, target_param_list):
        
        # initialisation en tant que Model keras
        super(Neural_network, self).__init__()

        # couche entrée + encodeur 
        self.var_param_list = var_param_list
        self.var = tf.keras.Sequential(self.var_param_list)
        
        # couche décodeur + sortie
        self.target_param_list = target_param_list
        self.target = tf.keras.Sequential(self.target_param_list)

    def call(self, x):
        
        var = self.var(x)
        target = self.target(var)
        
        return target

In [ ]:
# initialisation des couches du réseau de neurone
nn = Neural_network([tfl.Dense(4, input_dim=4, activation='sigmoid')], # nombre de neurones = nombre de features (input = x)]
                     [tfl.Dense(2, activation='softmax')]) # nombre de neurones = nombre de classes (output = 3)


# définition de l'optimizer
learning_rate = 3e-5 # learning rate
learning_rate = 0.01 # learning rate
loss = 'binary_crossentropy' # définition de la fonction de perte
loss = 'categorical_crossentropy'
optimizer = Adam(learning_rate=learning_rate)
metrics = ['accuracy']
nn.compile(optimizer=optimizer, loss=loss, metrics = metrics)

In [ ]:
# fit du modèle
epochs = 50
history = nn.fit(
    x=X_train2classes_norm, y=y_train2classes_norm, 
    epochs=200, 
    batch_size=10)

Epoch 1/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6847 - accuracy: 0.5622
Epoch 2/200
21/21 [==============================] - 0s 1ms/step - loss: 0.6381 - accuracy: 0.6617
Epoch 3/200
21/21 [==============================] - 0s 1ms/step - loss: 0.6003 - accuracy: 0.7363
Epoch 4/200
21/21 [==============================] - 0s 2ms/step - loss: 0.5697 - accuracy: 0.7612
Epoch 5/200
21/21 [==============================] - 0s 2ms/step - loss: 0.5352 - accuracy: 0.7662
Epoch 6/200
21/21 [==============================] - 0s 2ms/step - loss: 0.5058 - accuracy: 0.7711
Epoch 7/200
21/21 [==============================] - 0s 2ms/step - loss: 0.4833 - accuracy: 0.7662
Epoch 8/200
21/21 [==============================] - 0s 2ms/step - loss: 0.4641 - accuracy: 0.7612
Epoch 9/200
21/21 [==============================] - 0s 2ms/step - loss: 0.4530 - accuracy: 0.7811
Epoch 10/200
21/21 [==============================] - 0s 2ms/step - loss: 0.4423 - accuracy: 0.7811
Epoch 11/

In [ ]:
score=nn.evaluate(X_test2classes_norm, y_test2classes_norm, verbose = 2)
score[1]

2/2 - 0s - loss: 0.3971 - accuracy: 0.8039 - 157ms/epoch - 78ms/step


0.8039215803146362